# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [5]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [ ]:
# !pip install langchain_community

In [8]:
# Install the required package
# %pip install langchain openai

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [11]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7, model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)


In [12]:
prompt = ChatPromptTemplate.from_template( #Write a query that would take a variable to describe any product
 """
    Describe the product {product} in a few sentences.
    The description should be in a professional tone and should include the product's main features and benefits.
 """
)

In [ ]:
chain = (prompt | llm) #Create a chain that takes the prompt and the LLM as input

In [16]:
product = "Apple iPhone 14 Pro Max"
chain.invoke({"product": product}) #Invoke the chain with the product variable

AIMessage(content='The Apple iPhone 14 Pro Max is a flagship smartphone that seamlessly combines cutting-edge technology with elegant design. It features a stunning 6.7-inch Super Retina XDR display, offering vibrant colors and exceptional clarity, making it ideal for both media consumption and productivity. Powered by the A16 Bionic chip, the device delivers unparalleled performance and efficiency, ensuring smooth multitasking and responsive gaming experiences. The advanced camera system, including a 48MP main sensor, allows for remarkable photography, even in low-light conditions, while ProRAW and ProRes video capabilities provide professionals with unmatched creative control. Additionally, the iPhone 14 Pro Max boasts enhanced battery life, 5G connectivity, and robust security features, making it a versatile choice for users seeking both sophistication and functionality in their mobile experience.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 161, '

## SimpleSequentialChain

In [17]:
from langchain.chains import SimpleSequentialChain

In [21]:
llm = ChatOpenAI(temperature=0.9,model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)

In [23]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    """
        Describe the product {product} in a few sentences.
        The description should be in a professional tone and should include the product's main features and benefits.
    """
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [25]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    """
    summarize the description of the product in a few sentences: {product_description} 
    """
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [26]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [27]:
overall_simple_chain.run(product)

C:\Users\ma0me\AppData\Local\Temp\ipykernel_22208\228193363.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_simple_chain.run(product)




> Entering new SimpleSequentialChain chain...
The Apple iPhone 14 Pro Max is a premium smartphone that combines cutting-edge technology with sleek design. Featuring a stunning 6.7-inch Super Retina XDR display, it delivers vibrant colors and exceptional clarity for an immersive viewing experience. The device is powered by the A16 Bionic chip, ensuring lightning-fast performance and efficiency for demanding applications and multitasking. With a sophisticated camera system that includes a 48MP main lens, advanced low-light capabilities, and ProRAW and ProRes video recording, users can capture professional-quality photos and videos. Additionally, the iPhone 14 Pro Max supports 5G connectivity, providing ultra-fast download speeds and seamless streaming. With its durable design, enhanced battery life, and robust security features, the iPhone 14 Pro Max is engineered to meet the needs of the most discerning users.
The Apple iPhone 14 Pro Max is a high-end smartphone featuring a 6.7-inch S

'The Apple iPhone 14 Pro Max is a high-end smartphone featuring a 6.7-inch Super Retina XDR display for vibrant visuals and clarity. It is powered by the A16 Bionic chip for fast performance and multitasking, and includes a sophisticated camera system with a 48MP main lens for professional-quality photography and videography. Supporting 5G connectivity for fast download speeds, the device also boasts a durable design, extended battery life, and advanced security features, making it ideal for discerning users.'

**Repeat the above twice for different products**

In [28]:
overall_simple_chain.run("Google Pixel 9a")



> Entering new SimpleSequentialChain chain...
The Google Pixel 9a is a cutting-edge smartphone designed to deliver an exceptional user experience through its combination of advanced technology and user-friendly functionality. Featuring a vibrant 6.1-inch OLED display, the Pixel 9a offers stunning visuals and rich colors, making it ideal for media consumption and everyday usage. Equipped with a powerful Google Tensor chip, it ensures smooth performance and efficient multitasking. The device boasts an impressive camera system, including advanced computational photography capabilities that allow users to capture stunning images in any lighting condition. Additionally, the Pixel 9a runs on the latest version of Android, providing seamless access to Google services and regular updates for enhanced security and performance. With its sleek design and robust features, the Google Pixel 9a stands out as a reliable choice for both productivity and entertainment.
The Google Pixel 9a is an advanc

'The Google Pixel 9a is an advanced smartphone that offers a vibrant 6.1-inch OLED display for stunning visuals and rich colors, making it perfect for media consumption. Powered by the Google Tensor chip, it delivers smooth performance and efficient multitasking. Its impressive camera system features advanced computational photography capabilities, allowing users to take exceptional photos in various lighting conditions. Running on the latest Android version, it provides easy access to Google services and regular updates, making it a reliable option for both productivity and entertainment.'

## SequentialChain

In [29]:
from langchain.chains import SequentialChain

In [43]:
llm = ChatOpenAI(temperature=0.9, model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)


first_prompt = ChatPromptTemplate.from_template(
  #This prompt should translate a review
  """
    Write an English review of the product {product} in a few sentences.
    """
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="firstPrompt" #Give a name to your output
                    )


In [44]:
second_prompt = ChatPromptTemplate.from_template(
    #Write a promplt to summarize a review
    """
    summarize the review of the product in a few sentences: {firstPrompt}
    """
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="secondPrompt" #give a name to this output
                    )


In [45]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
   """
    Translate the review to Arabic: {secondPrompt}\nMake sure to answer with the translation only. ONLY THE TRANSLATION.
    """
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="thirdPrompt" #give a name to this output
                      )


In [46]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
       """
Using the outputs of the previous two prompts, craft a follow-up message. 
The first output is: {secondPrompt}
The second output is: {thirdPrompt}
       """
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="frouthPrompt" #give a name to this output
                     )


In [47]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["product"],
    output_variables=["secondPrompt", "thirdPrompt", "frouthPrompt"],
    verbose=True
)

In [48]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'product': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'secondPrompt': "The reviewer was disappointed with the product's quality, noting that the flavor was mediocre and lacked the expected richness. They found the mousse's consistency unsatisfactory and compared it unfavorably to similar items purchased elsewhere. Suspecting it might be an old batch or counterfeit, the reviewer ultimately does not recommend the product.",
 'thirdPrompt': 'كان المراجع مخيبًا للأمل بجودة المنتج، مشيرًا إلى أن النكهة كانت متوسطة ويفتقر إلى الغنى المتوقع. وجدوا أن قوام الموس غير مرضٍ وقارنه بسلبية مع عناصر مماثلة تم شراؤها من أماكن أخرى. مشددًا على أنه قد يكون دفعة قديمة أو مزيفة، في النهاية لم يوصِ المراجع بالمنتج.',
 'frouthPrompt': "Subject: Follow-Up on Product Quality Concerns\n\nDear [Reviewer’s Name],\n\nThank you for sharing your feedback regarding your recent experience w

**Repeat the above twice for different products or reviews**

In [50]:
review = df.Review[4]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'product': "\xa0I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck.",
 'secondPrompt': 'The reviewer initially loved the frother for its performance, but was disappointed to encounter a recurring issue where the frother detaches from the handle after just a few months of use. Despite positive customer service experiences with replacements, the second frother also failed after four months. Due to these durability concerns, the reviewer is now seeking a more reliable alternative.',
 'thirdPrompt': 'أحب المراجع في البداية جهاز الرغوة لأدائه، لكنه شعر بخيبة أمل عندما واجه مشكلة متكررة تتمثل في انفصال جهاز الرغوة عن المقبض بعد بضع أ

In [51]:
review = df.Review[3]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'product': 'This is the best throw pillow fillers on Amazon. I’ve tried several others, and they’re all cheap and flat no matter how much fluffing you do. Once you toss these in the dryer after you remove them from the vacuum sealed shipping material, they fluff up great',
 'secondPrompt': "The reviewer highly praises the throw pillow fillers purchased from Amazon, stating they are the best they've tried. Unlike previous fillers that remained flat, these fluff up beautifully after a quick dryer toss, providing excellent support and comfort. The reviewer recommends these fillers for anyone looking to enhance their throw pillows, noting that they make the pillows look full and inviting.",
 'thirdPrompt': 'المراجع يمدح بشدة حشوات الوسائد الرمي التي تم شراؤها من أمازون، مشيرًا إلى أنها الأفضل التي جربها. على عكس الحشوات السابقة التي ظلت مسطحة، فإن هذه الحشوات تنتفخ بشكل جميل بعد رميها سريعًا في المجفف، مما يوفر دعمًا وراحة ممتازة. يوصي المراجع بهذه الحشوات لأي شخص يبحث عن تحسين وسائد الرم

## Router Chain

In [52]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [53]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [54]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [55]:
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

In [56]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [57]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [58]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [59]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [60]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\ma0me\AppData\Local\Temp\ipykernel_22208\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [61]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by an idealized object known as a "black body," which absorbs all incident radiation, regardless of frequency or angle. A perfect black body is a theoretical concept that does not reflect or transmit any light, making it appear completely black at room temperature.\n\nWhen a black body is heated, it emits radiation in a characteristic spectrum that depends solely on its temperature. This emission can be described by Planck\'s law, which shows that the intensity of radiation emitted at different wavelengths increases with temperature. As the temperature rises, the peak of the emitted spectrum shifts to shorter wavelengths, a phenomenon known as Wien\'s displacement law.\n\nBlack body radiation played a crucial role in the development of quantum mechanics, particularly through Max Planck\'s work in the early 20th century, which introduced the idea of quantized energy levels. This helped resolve the "ultraviolet catastr

In [62]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'To break down the problem \\(2 + 2\\):\n\n1. Identify the numbers involved: We have the numbers 2 and 2.\n2. Understand the operation: The operation is addition, which means we are combining the two numbers.\n\nNow, we can perform the addition:\n\n\\[\n2 + 2 = 4\n\\]\n\nSo, the answer to the question \\(2 + 2\\) is \\(4\\).'

In [63]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA serves as the fundamental blueprint for life. Here are several key reasons why this is the case:\n\n1. **Genetic Information Storage**: DNA (deoxyribonucleic acid) contains the genetic instructions necessary for the development, functioning, growth, and reproduction of all living organisms. It encodes the information required to produce proteins, which perform a vast array of functions within the cell.\n\n2. **Cellular Function and Identity**: Each cell in our body has a specific role, whether it be a muscle cell, nerve cell, or skin cell. The DNA in each cell contains the complete set of instructions for the organism, but different cells express different genes. This selective gene expression allows cells to specialize and perform their unique functions while still retaining the complete genetic information.\n\n3. **Replication and Inheritance**: DNA is capable of self-replication, which is essential for cell division. When a cell divid

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [64]:
chain.run("What is the contents of RAM in a computer?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'What is the content of RAM in a computer?'}
> Finished chain.


'The content of RAM (Random Access Memory) in a computer consists of data and instructions that the CPU (Central Processing Unit) needs to access quickly while performing tasks. Here’s a breakdown of what this content typically includes:\n\n1. **Operating System**: The core components of the operating system (OS) that are currently in use, allowing the CPU to manage hardware and software resources.\n\n2. **Application Data**: Data from applications that are currently running. This can include user data, temporary files, and any other information that the application needs to function.\n\n3. **Program Instructions**: The executable code of programs that are currently being executed. This includes the instructions that the CPU processes to perform tasks.\n\n4. **Cache Data**: Frequently accessed data that is stored in RAM to speed up processing times. This can include variables, arrays, and other data structures that are used by running applications.\n\n5. **Buffers**: Temporary storage 